# Why do we need Seeding:

https://stackoverflow.com/questions/21494489/what-does-numpy-random-seed0-do

In [3]:
from sklearn.metrics import accuracy_score
import xgboost as xgb
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint

from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from scipy.stats import pearsonr
from math import sqrt
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix

import warnings



np.random.seed(0)
warnings.filterwarnings("ignore")







test_df = pd.read_csv(r'test_dataset.csv')
train_df = pd.read_csv(r'balanced_train_dataset.csv')




label = ['Analysis','Backdoor','DoS','Exploits','Fuzzers','Generic','Normal','Reconnaissance','Shellcode','Worms']

X_train = train_df.iloc[:,:train_df.shape[1]-10]
Y_train = train_df.iloc[:,-10:].to_numpy()

X_test = test_df.iloc[:,:test_df.shape[1]-10]
Y_test = test_df.iloc[:,-10:].to_numpy()



def LabelAttack(Y):
  attack = []
  for i in range(Y.shape[0]):
    k = 0
    for j in range(Y.shape[1]):
      if(Y[i][j] == 1):
        k = j
        break
    attack.append(k)
  return attack


Y_train = np.asarray(LabelAttack(Y_train)).reshape(-1,1)
Y_test = np.asarray(LabelAttack(Y_test)).reshape(-1,1)



print("Number of training samples: ",train_df.shape[0])
print("Number of testing samples: ",test_df.shape[0])
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

Number of training samples:  363144
Number of testing samples:  82332
(363144, 44)
(363144, 1)
(82332, 44)
(82332, 1)


# Maine alag se likha hai

In [4]:
import pickle


def get_model():
    return XGBClassifier(learning_rate = 0.1,
          n_estimators=8, 
          max_depth=3,
          nthread=4,
          seed = 1)

def save_generation(gen_id, constant, data):
    with open(str(gen_id) + "_" + str(constant) + "_.pickle", "wb") as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
        

def load_generation(gen_id, constant):
    with open(str(gen_id) + "_" + str(constant) + "_.pickle", 'rb') as handle:
        data = pickle.load(handle)

    return data



model = get_model()

In [5]:



def AnovaScores(X_train, y_train):
	# configure to select all features
	fs = SelectKBest(score_func=f_classif, k='all')
	# learn relationship from training data
	fs.fit(X_train, y_train)
	# transform train input data
	X_train_fs = fs.transform(X_train)
	return fs.scores_


score = AnovaScores(X_train,Y_train)
score = np.asarray(score).reshape(len(score),-1)

#Scale AnovaScore
mimxScale = MinMaxScaler()
score = mimxScale.fit_transform(score)







def countNoOfFeatures(chromosome):
    k = 0
    for gene in chromosome:
      k += gene
    return k

def computeMerit(chromosome):
    print("computing merit:")
    k = countNoOfFeatures(chromosome)
    x = X_train.iloc[:,chromosome].to_numpy()
    y = Y_train

    n = x.shape[1]
    Rcf = 0
    Rff = 0
    for i in range(n):
      for j in range(i+1,n):
        Rff += pearsonr(x[:,i],x[:,j])[0]
    
    Rff = Rff/((n*(n-1))/2)
    Rcf = 0.0
    for i in range(score.shape[0]):
      if(chromosome[i]==1):
        Rcf += score[i][0]
    Rcf = Rcf/n

    print("Rcf: ",Rcf,"/","Rff: ",Rff)

    merit = (k*Rcf)/sqrt(k+k*(k-1)*abs(Rff))
    return merit


def GMean(y_pred,y_true):
    TP,TN,FP,FN = 0,0,0,0
    cnf_matrix = confusion_matrix(y_true,y_pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    sensitivity = TP/(TP+FN) #true positive rate
    specificity = TN/(TN+FP) #true negative rate
    
    g_mean = sqrt((sum(sensitivity)/len(sensitivity)) * (sum(specificity)/len(specificity)))
    return g_mean


def initilization_of_population(size,n_feat):
    population = []
    for i in range(size):
        chromosome = np.ones(n_feat,dtype=np.bool)     
        chromosome[:int(0.3*n_feat+0.5)]=False             
        np.random.shuffle(chromosome)
        population.append(chromosome)
    return population

def fitness_score(population, generation, a):
    scores = []
   

    #varibale declare kiya
    merits = []
    g_means = []
    f_scores = []
    accs = []
    
    
    for chromosome in population:
        merit = computeMerit(chromosome)
        model.fit(X_train.iloc[:,chromosome],Y_train)         
        predictions = model.predict(X_test.iloc[:,chromosome])
        g_mean = GMean(predictions,Y_test)
        f_score = (a * merit) + ((1-a)*g_mean)
        acc = accuracy_score(Y_test,predictions)

        scores.append(f_score)

        #variable me value append kari
        merits.append(merit)
        g_means.append(g_mean)
        f_scores.append(f_score)
        accs.append(acc)


    #variable se dictionry me convert kiya
    fitness_metrics = {
          'merit' : merits,
           'g_mean' : g_means,
           'f_score' : f_scores,
           'acc' : accs
    }
    

    scores, population = np.array(scores), np.array(population) 
    inds = np.argsort(scores)                                             ###sorting in ascending order
    
    #also returning fitness matrics
    return list(scores[inds][::-1]), list(population[inds,:][::-1]), fitness_metrics 


def selection(pop_after_fit,selection_rate,pop_size):
    population_nextgen = []
    count = int(selection_rate*pop_size+0.5)                     
    for i in range(count):
        population_nextgen.append(pop_after_fit[i])
    return population_nextgen
    

def crossover(pop_after_sel,selection_rate,pop_size):
    print("Crossover operation: ")
    pop_nextgen = pop_after_sel
    count = (int((1-selection_rate)*pop_size + 0.5))//2
    for i in range(0,count,2):
        new_par1 = []
        new_par2 = []
        child_1 , child_2 = pop_nextgen[i] , pop_nextgen[i+1]
        new_par1 = np.concatenate((child_1[:len(child_1)//2],child_2[len(child_1)//2:]))
        new_par2 = np.concatenate((child_1[len(child_1)//2:],child_2[:len(child_1)//2]))
        pop_nextgen.append(new_par1)
        pop_nextgen.append(new_par2)
    return pop_nextgen


def mutation(pop_after_cross,mutation_rate,n_feat):   
    print("Mutation operation: ")
    mutation_range = int(mutation_rate*n_feat+0.5)
    pop_next_gen = []
    for n in range(0,len(pop_after_cross)):
        chromo = pop_after_cross[n]
        rand_posi = [] 
        for i in range(0,mutation_range):
            #pos = randint(0,n_feat-1)
            pos = np.random.randint(n_feat-1, size=1)[0]
            
            rand_posi.append(pos)
        for j in rand_posi:
            chromo[j] = not chromo[j]  
        pop_next_gen.append(chromo)
    return pop_next_gen



In [9]:
def generations(pop_size,n_feat,mutation_rate,selection_rate, a, n_gen, from_gen = 0):
    best_chromo= []
    best_score= []
    population_nextgen=initilization_of_population(pop_size,n_feat)
    
    #----------------------Ye maine change kiya------#
    
    scores = None
    pop_after_fit = None
    fitness_metrics = None
    pop_after_sel = None
    pop_after_cross = None

    if from_gen != 0:
        
        print(f"Need to load from prior run generation : {from_gen}")
        _generation_data = load_generation(gen_id = from_gen, constant = a)

        scores = _generation_data["scores"]
        pop_after_fit = _generation_data["pop_after_fit"]
        fitness_metrics = _generation_data["fitness_metrics"]
        pop_after_sel = _generation_data["pop_after_sel"]
        pop_after_cross = _generation_data["pop_after_cross"]
        population_nextgen = _generation_data["population_nextgen"]
#       best_chromo = _generation_data["best_chromo"]
#       best_score = _generation_data["best_score"]

        from_gen += 1
    #----------------------Yaha tak------#
    
    
    for i in range(from_gen, n_gen):
        scores, pop_after_fit, fitness_metrics = fitness_score(population_nextgen, generation = i, a = a)
        print('Best score in generation',i,':',scores[:1])  #2
        
        pop_after_sel = selection(pop_after_fit,selection_rate,pop_size) 
        pop_after_cross = crossover(pop_after_sel,selection_rate,pop_size)
        population_nextgen = mutation(pop_after_cross,mutation_rate,n_feat)
        best_chromo.append(pop_after_fit[0])
        best_score.append(scores[0])
        
        
        
        print("\n==============================================================\n")
        print(f"saving generation : {i}")
        _generation_data = {
            "scores": scores,
            "pop_after_fit" : pop_after_fit,
            "fitness_metrics" : fitness_metrics,
            "pop_after_sel" : pop_after_sel,
            "pop_after_cross" : pop_after_cross,
            "population_nextgen" : population_nextgen,
            "best_chromo" : pop_after_fit[0],
            "best_score" : scores[0],   
        }
        
        
        save_generation(gen_id = i, constant = a, data = _generation_data)
        print(f"generation : {i} : constant : {a} saved successfully!")
        
    
        if gen_id in [7, 15, 31, 63, 95, 127]:
            X_train_df, Y_train_df, X_test_df, Y_test_df = save_generated_feats(gen_id, score_bc, chromo_df_bc)
            train_models(gen_id, X_train_df, Y_train_df, X_test_df, Y_test_df)
            
    return best_chromo,best_score




In [10]:


chromo_df_bc,score_bc = generations(pop_size=10,n_feat=X_train.shape[1],mutation_rate=0.20,selection_rate=0.8,a = 0.4, n_gen=128, from_gen=0)
save_generated_feats(128, score_bc, chromo_df_bc)

computing merit:
Rcf:  0.032258453063504104 / Rff:  0.02199137718774412
[09:54:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.032258494777117265 / Rff:  0.01994413477098124
[09:55:22] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  4.200992949548073e-07 / Rff:  0.032358688961965795
[09:56:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  

computing merit:
Rcf:  0.038462056099715376 / Rff:  0.02731964631652452
[10:10:42] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  3.895693007661326e-07 / Rff:  0.04595866714239843
[10:11:19] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.04000050643765621 / Rff:  0.02206702168720399
[10:11:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.

computing merit:
Rcf:  0.050000202479532796 / Rff:  0.02810371354713031
[10:25:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.04347852949171512 / Rff:  0.04257342624666336
[10:25:37] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best score in generation 4 : [0.4628951265907327]
Crossover operation: 
Mutation operation: 


saving generation : 4
generation : 4 : constant : 0.4 saved successfully!
computing merit:
Rcf:  0.050000127243880935 / Rff:  0.017034428847335605
[10:26:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used wit

Rcf:  0.041666996916256194 / Rff:  0.021141529895717326
[10:39:06] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.04000046550668605 / Rff:  0.04154380050581349
[10:39:45] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.04545475432393556 / Rff:  0.009224479932021452
[10:40:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  0.04347848370173889

computing merit:
Rcf:  0.0384619540182855 / Rff:  0.017550801520472215
[10:48:57] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  3.816575576280056e-07 / Rff:  0.029673474540942782
[10:49:13] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf:  3.151265307766672e-07 / Rff:  0.011050260875813305
[10:49:36] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
computing merit:
Rcf: 

In [1]:

def save_generated_feats(gen_id, score_bc, chromo_df_bc):
    ind = 0
    l = len(score_bc)
    _max = 0

    for i in range(l):
      if(_max < score_bc[i]):
        _max = score_bc[i]
        ind = i


    chromosome = chromo_df_bc[ind]

    print(chromosome)


    X_train_df = X_train.iloc[:, chromosome]
    X_test_df = X_test.iloc[:, chromosome]


    Y_train_df = pd.DataFrame(Y_train, columns =['attack_cat'])
    Y_test_df = pd.DataFrame(Y_test, columns = ['attack_cat'])

    train_df = pd.concat([X_train_df, Y_train_df] , axis = 1)
    test_df = pd.concat([X_test_df, Y_test_df] , axis = 1)




    train_df.to_csv(f"train_df_after_fs_{gen_id}.csv",index=False)
    test_df.to_csv(f"test_df_after_fs_{gen_id}.csv",index = False)
    
    return X_train_df, Y_train_df, X_test_df, Y_test_df

In [ ]:

import numpy as np


import modin.pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, cross_val_score
from scipy.stats import pearsonr
from math import sqrt

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix


import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from random import randint
%matplotlib inline 
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score


from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score


from xgboost.sklearn import XGBClassifier
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

import pickle
from joblib import parallel_backend


def save_gen_model_res(gen_id, data):
    with open(f'gen_model_res_{str(gen_id)}_.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_gen_model_res(gen_id) :    
    with open(f'gen_model_res_{str(gen_id)}_.pickle', 'rb') as handle:
        return pickle.load(handle)
    
    
    




def DirectMetrics(actual, predicted):
    
    cr = classification_report(actual, predicted)
    print("Classification_Report : ")
    print(cr)
  
    Accuracy = accuracy_score(actual, predicted)
    print("Accuracy: %.2f%%" % (Accuracy * 100.0))
    
    Precision = precision_score(actual, predicted, average = 'macro')
    Recall = recall_score(actual, predicted, average = 'macro')
    F1_Score = f1_score(actual, predicted, average = 'macro')
    
    return {"Direct_Precision" : Precision,
           "Direct_Recall" : Recall,
           "Direct_F1-Score" : F1_Score}


def ComputeMetrics(actual , predicted):
    TP, TN, FP, FN = 0, 0, 0, 0
    
    cm = confusion_matrix(actual, predicted)
    
    plt.figure(figsize = (9,9))
    sns.heatmap(cm, annot = True, fmt = '0.3f', linewidth = 0.5, square = True, cbar = True)
    plt.xlabel('Predicted Labels')
    plt.ylabel('Actual Labels')
    plt.title('Confusion Matrix')
    
    FP = cm.sum(axis = 0) - np.diag(cm)
    FN = cm.sum(axis = 1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)

    sensitivity = TP/(TP+FN)
    avg_sensitivity = sum(sensitivity)/len(sensitivity)

    specificity = TN/(TN+FP)
    avg_specificity = sum(specificity)/len(specificity)

    precision = TP/(TP+FP)
    avg_precision = sum(precision)/len(precision)
    
    recall = TP/(TP+FN)
    avg_recall = sum(recall)/len(recall)
    
    f1_score = (2*recall*precision)/(recall + precision)
    avg_f1_score = sum(f1_score)/len(f1_score)
    
    return {"Sensitivity " : avg_sensitivity,
           "Specificity " : avg_specificity,
           "Precision " : avg_precision,
           "Recall " : avg_recall,
           "F1_Score " : avg_f1_score}          

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

    
    
def train_models(gen_id, X_train_df, Y_train_df, X_test_df, Y_test_df):

    _dict = { model_type : {"direct_metrics" : None, "metrics" : None} for model_tye in ["lr", "dt", "xgb"]}

    #Logistic Regression
    model_lr = LogisticRegression()
    model_lr.fit(X_train_df, Y_train_df)

    Y_pred_lr = model_lr.predict(X_test_df)
    predictions_lr = [round(value) for value in Y_pred]

    direct_metrics_lr = DirectMetrics(Y_test_df, predictions_lr)
    metrics_lr = ComputeMetrics(Y_test_df, predictions_lr)





    #Decision Tree
    model_dt = DecisionTreeClassifier()
    model_dt.fit(X_train_df, Y_train_df)

    Y_pred_dt = model.predict(X_test_df)
    predictions_dt = [round(value) for value in Y_pred_dt]

    direct_metrics_dt = DirectMetrics(Y_test_df, predictions_dt)
    metrics_dt = ComputeMetrics(Y_test_df, predictions_dt)




    #XGBoost
    model_xgb = XGBClassifier()
    model_xgb.fit(X_train_df, Y_train_df)

    Y_pred_xgb = model.predict(X_test_df)
    predictions_xgb = [round(value) for value in Y_pred_xgb]

    direct_metrics_xgb = DirectMetrics(Y_test_df, predictions_xgb)
    metrics_xgb = ComputeMetrics(Y_test_df, predictions_xgb)


    _dict["lr"]["direct_metrics"] = direct_metrics_lr
    _dict["lr"]["metrics"] = metrics_lr

    _dict["dt"]["direct_metrics"] = direct_metrics_dt
    _dict["dt"]["metrics"] = metrics_dt

    _dict["xgb"]["direct_metrics"] = direct_metrics_xgb
    _dict["xgb"]["metrics"] = metrics_xb
    
    #save the values
    save_gen_model_res(gen_id, _dict)
    print("*" * 50)
    print("*" * 50)
    print(_dict)
    print("*" * 50)
    
    



